In [1]:
import numpy as np
import pandas as pd
import json
from flask import Flask, jsonify
from flask_cors import CORS
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as XGB

In [2]:
# Read in the tuned hyperparameters
try:
    # Open the JSON file
    with open('Models.json', 'r') as file:
        # Load the models
        models = json.load(file)
    # File is successfully opened and loaded
    print(models)
except FileNotFoundError:
    # Handle the FileNotFoundError
    print("The file 'Models.json' does not exist or cannot be opened.")

[[{'C': 1000, 'penalty': 'l1', 'solver': 'saga'}, {'C': 10, 'penalty': 'l2', 'solver': 'saga'}, {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l1', 'solver': 'saga'}], [{'booster': 'gbtree', 'learning_rate': 0.1, 'n_estimators': 100, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'max_depth': 6, 'min_child_weight': 10, 'gamma': 0.1, 'subsample': 1.0, 'colsample_bytree': 0.8, 'lambda': None, 'alpha': None}, {'booster': 'gbtree', 'learning_rate': 0.1, 'n_estimators': 100, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'max_depth': 6, 'min_child_weight': 10, 'gamma': 0.1, 'subsample': 1.0, 'colsample_bytree': 0.8, 'lambda': None, 'alpha': None}, {'booster': 'gbtree', 'learning_rate': 0.1, 'n_estimators': 100, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'max_depth': 6, 'min_child_weight': 10, 'gamma': 0.1, 'subsample': 1.0, 'colsample_bytree': 0.8, 'lambda': None, 'alpha': None}, {'booster': 'gbtree', 'learning_rate': 0.1, 'n_est

In [3]:
# Read in the list of columns that were not used to train
try:
    # Open the JSON file
    with open('FeatureDropped.json', 'r') as file:
        # Load the models
        features_dropped = json.load(file)
    # File is successfully opened and loaded
    print(features_dropped)
except FileNotFoundError:
    # Handle the FileNotFoundError
    print("The file 'FeatureDropped.json' does not exist or cannot be opened.")

['CustomerID', 'Gender', 'ChurnDate', 'FirstPersona', 'SecondPersona', 'ThirdPersona', 'CombinedPersonas', 'CurrLifecycle', 'SocialInfluencer', 'CurrLifecycle_Active', 'CurrLifecycle_Reactivated', 'CurrLifecycle_Churned', 'CurrLifecycle_Dormant', 'PrevLifecycle_Active', 'PrevLifecycle_Churned', 'PrevLifecycle_Dormant', 'PrevLifecycle_Reactivated']


In [29]:
# Read in the train and test data which are the original customer base split into train and test
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Read in the oversampled train data for the fitting of models
# The oversampling is done from train.csv
train_data_oversampled = pd.read_csv('train_oversampled.csv')
input_features = [column for column in train_data.columns if column not in features_dropped]

In [5]:
# Encode categorical variables for logistic regression
label_encoder = LabelEncoder()
train_data_encoded = train_data.copy()
test_data_encoded = test_data.copy()
train_data_oversampled_encoded = train_data_oversampled.copy()

train_data_encoded['MonthsInactive'] = train_data_encoded['MonthsInactive'].fillna(0)
test_data_encoded['MonthsInactive'] = test_data_encoded['MonthsInactive'].fillna(0)
train_data_oversampled_encoded['MonthsInactive'] = train_data_oversampled_encoded['MonthsInactive'].fillna(0)

for column in train_data_encoded.columns:
    if train_data_encoded[column].dtype == 'object':
        train_data_encoded[column] = label_encoder.fit_transform(train_data_encoded[column])
        test_data_encoded[column] = label_encoder.fit_transform(test_data_encoded[column])
        train_data_oversampled_encoded[column] = label_encoder.fit_transform(train_data_oversampled_encoded[column])

In [6]:
# Copy data sets to do encoding and scaling for logistic regression
train_lgr = train_data_encoded.copy()
test_lgr = test_data_encoded.copy()
train_oversampled_lgr = train_data_oversampled_encoded.copy()

In [7]:
# Scale dataset (except for the labels) for logistic regression
scaler = StandardScaler()

#label_columns = ['CurrLifecycle_Active', 'CurrLifecycle_Reactivated', 'CurrLifecycle_Dormant', 'CurrLifecycle_Churned']
#feature_columns = [column for column in train_lgr.columns if column not in label_columns]

train_lgr_features = train_lgr[input_features]
#train_lgr_labels = train_lgr[label_columns]

train_lgr_features = pd.DataFrame(scaler.fit_transform(train_lgr_features), columns = train_lgr_features.columns)
#train_lgr = pd.concat([train_lgr_features, train_lgr_labels], axis = 1)

test_lgr_features = test_lgr[input_features]
#test_lgr_labels = test_lgr[label_columns]

test_lgr_features = pd.DataFrame(scaler.fit_transform(test_lgr_features), columns = test_lgr_features.columns)
#test_lgr = pd.concat([test_lgr_features, test_lgr_labels], axis = 1)

train_oversampled_lgr_features = train_oversampled_lgr[input_features]
#train_oversampled_lgr_labels = train_oversampled_lgr[label_columns]

train_oversampled_lgr_features = pd.DataFrame(scaler.fit_transform(train_oversampled_lgr_features), columns = train_oversampled_lgr_features.columns)
#train_oversampled_lgr = pd.concat([train_oversampled_lgr_features, train_oversampled_lgr_labels], axis = 1)

In [8]:
# Separate the lgr, xgb, and rf parameters into
# a list of 4 dictionaries with key value pairs of hyperparamters for Active, Reactivated, Dormant, and Churned Classification
lgr = models[0]
xgb = models[1]
rf = models[2]

In [9]:
y_train_oversampled_all_lgr = [train_oversampled_lgr['CurrLifecycle_Active'], train_oversampled_lgr['CurrLifecycle_Reactivated'], train_oversampled_lgr['CurrLifecycle_Dormant'], train_oversampled_lgr['CurrLifecycle_Churned']]
#y_test_encoded_all_lgr = [test_lgr['CurrLifecycle_Active'], test_lgr['CurrLifecycle_Reactivated'], test_lgr['CurrLifecycle_Dormant'], test_lgr['CurrLifecycle_Churned']]

y_train_oversampled_all = [train_data_oversampled_encoded['CurrLifecycle_Active'], train_data_oversampled_encoded['CurrLifecycle_Reactivated'], train_data_oversampled_encoded['CurrLifecycle_Dormant'], train_data_oversampled_encoded['CurrLifecycle_Churned']]
#y_test_all = [test_data_encoded['CurrLifecycle_Active'], test_data_encoded['CurrLifecycle_Reactivated'], test_data_encoded['CurrLifecycle_Dormant'], test_data_encoded['CurrLifecycle_Churned']]

In [10]:
lgr_models_list = []
xgb_models_list = []
rf_models_list = []
for i in range(4):
    # Train the lgr models and store in the list
    lgr_model = LogisticRegression(**lgr[i])
    lgr_model.fit(train_oversampled_lgr_features, y_train_oversampled_all_lgr[i])
    lgr_models_list.append(lgr_model)
    
    # Train the xgb models and store in the list
    xgb_model = XGB.XGBClassifier(**xgb[i])
    xgb_model.fit(train_data_oversampled_encoded[input_features], y_train_oversampled_all[i])
    xgb_models_list.append(xgb_model)
    
    # Train the rf models and store in the list
    rf_model = RandomForestClassifier(**rf[i])
    rf_model.fit(train_data_oversampled_encoded[input_features], y_train_oversampled_all[i])
    rf_models_list.append(rf_model)

C:\Users\ericn\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ericn\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\ericn\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ericn\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [11]:
# Define the function to generate predictions from the trained models
def prediction(lgr, xgb, rf, data, features):
    # Generate predictions for data
    for i in range(len(lgr)):
        # lgr[0], xgb[0], rf[0] is the model with binary label Active/Non-Active
        if i == 0:
            data['lgr_Active_proba'] = lgr[0].predict_proba(data[features])[:, 1]
            data['xgb_Active_proba'] = xgb[0].predict_proba(data[features])[:, 1]
            data['rf_Active_proba'] = rf[0].predict_proba(data[features])[:, 1]
            
        # lgr[1], xgb[1], rf[1] is the model with binary label Reactivated/Non-Reactivated
        elif i == 1:
            data['lgr_Reactivated_proba'] = lgr[1].predict_proba(data[features])[:, 1]
            data['xgb_Reactivated_proba'] = xgb[1].predict_proba(data[features])[:, 1]
            data['rf_Reactivated_proba'] = rf[1].predict_proba(data[features])[:, 1]
            
        # lgr[2], xgb[2], rf[2] is the model with binary label Dormant/Non-Dormant
        elif i == 2:
            data['lgr_Dormant_proba'] = lgr[2].predict_proba(data[features])[:, 1]
            data['xgb_Dormant_proba'] = xgb[2].predict_proba(data[features])[:, 1]
            data['rf_Dormant_proba'] = rf[2].predict_proba(data[features])[:, 1]
           
        # lgr[3], xgb[3], rf[3] is the model with binary label Churned/Non-Churned
        elif i == 3:
            data['lgr_Churned_proba'] = lgr[3].predict_proba(data[features])[:, 1]
            data['xgb_Churned_proba'] = xgb[3].predict_proba(data[features])[:, 1]
            data['rf_Churned_proba'] = rf[3].predict_proba(data[features])[:, 1]
    
    # Calculate the average probability from the probabilities generated by each model (Ensemble Learning)
    data['average_Active_proba'] = data[['lgr_Active_proba', 'xgb_Active_proba', 'rf_Active_proba']].mean(axis = 1)
    data['average_Reactivated_proba'] = data[['lgr_Reactivated_proba', 'xgb_Reactivated_proba', 'rf_Reactivated_proba']].mean(axis = 1)
    data['average_Dormant_proba'] = data[['lgr_Dormant_proba', 'xgb_Dormant_proba', 'rf_Dormant_proba']].mean(axis = 1)
    data['average_Churned_proba'] = data[['lgr_Churned_proba', 'xgb_Churned_proba', 'rf_Churned_proba']].mean(axis = 1)
    
    # Based on the definition of lifecycle, it is not possible for a customer to have the below stated transitions
    # Active -> Reactivated, Dormant -> Dormant, Dormant -> Active, Reactivated -> Reactivated
    # Hence, set the probabilities of these cases to 0
    data['average_Active_proba'] = np.where((data['CurrLifecycle_Dormant'] == 1) & (data['average_Active_proba'] > 0), 0, data['average_Active_proba'])
    data['average_Reactivated_proba'] = np.where(((data['CurrLifecycle_Active'] == 1) | (data['CurrLifecycle_Reactivated'] == 1)) & (data['average_Reactivated_proba'] > 0), 0, data['average_Reactivated_proba'])
    data['average_Dormant_proba'] = np.where((data['CurrLifecycle_Dormant'] == 1) & (data['average_Dormant_proba'] > 0), 0, data['average_Dormant_proba'])
    
    # Normalize the probablities such that each row adds up to 1
    total_prob = data[['average_Active_proba', 'average_Reactivated_proba', 'average_Dormant_proba', 'average_Churned_proba']].sum(axis=1)
    data['average_Active_proba'] = data['average_Active_proba'] / total_prob
    data['average_Reactivated_proba'] = data['average_Reactivated_proba'] / total_prob
    data['average_Dormant_proba'] = data['average_Dormant_proba'] / total_prob
    data['average_Churned_proba'] = data['average_Churned_proba'] / total_prob
    
    # Rename the probabilities column
    data['Active'] = data['average_Active_proba']
    data['Reactivated'] = data['average_Reactivated_proba']
    data['Dormant'] = data['average_Dormant_proba']
    data['Churned'] = data['average_Churned_proba']
    
    # The lifecycle with the highest probability will be the predicted lifecycle
    data['PredictedLifecycle'] = data[['Active', 'Reactivated', 'Dormant', 'Churned']].idxmax(axis=1)

    return data

In [12]:
# Filter the existing active customers from the train_data
#train_drop_index = train_data_encoded[(train_data_encoded['CurrLifecycle_Dormant'] == 1) | (train_data_encoded['CurrLifecycle_Churned'] == 1)].index
#train_active = train_data_encoded.drop(train_drop_index)
train_active = train_data_encoded[train_data_encoded['CurrLifecycle_Churned'] != 1].copy()
#train_data = train_data.drop(train_drop_index)

# Returns the df with predictions for train_active (This train data only consists of existing active customers)
# Note: train_active_prediction is the encoded dataframe
train_active_prediction = prediction(lgr_models_list, xgb_models_list, rf_models_list, train_active, input_features)

# Returns the df with predictions for test_data 
# This test data consists of both active and churned customers to generate classification report for model performance evaluation
# Note: test_prediction is the encoded dataframe
test_prediction = prediction(lgr_models_list, xgb_models_list, rf_models_list, test_data_encoded, input_features)

In [13]:
# Add the probabilities and predicted Lifecycle columns to the non-encoded dataframe
train_data = pd.concat([train_data, train_active_prediction.loc[:, 'average_Active_proba':'average_Churned_proba'], train_active_prediction['PredictedLifecycle']], axis=1)
test_data = pd.concat([test_data, test_prediction.loc[:, 'average_Active_proba':'average_Churned_proba'], test_prediction['PredictedLifecycle']], axis=1)

In [14]:
# Encode the Predicted Lifecycles
test_data['PredLifecycle_Active'] = (test_data['PredictedLifecycle'] == 'Active').astype(int)
test_data['PredLifecycle_Reactivated'] = (test_data['PredictedLifecycle'] == 'Reactivated').astype(int)
test_data['PredLifecycle_Dormant'] = (test_data['PredictedLifecycle'] == 'Dormant').astype(int)
test_data['PredLifecycle_Churned'] = (test_data['PredictedLifecycle'] == 'Churned').astype(int)

train_data['PredLifecycle_Active'] = (train_data['PredictedLifecycle'] == 'Active').astype(int)
train_data['PredLifecycle_Reactivated'] = (train_data['PredictedLifecycle'] == 'Reactivated').astype(int)
train_data['PredLifecycle_Dormant'] = (train_data['PredictedLifecycle'] == 'Dormant').astype(int)
train_data['PredLifecycle_Churned'] = (train_data['PredictedLifecycle'] == 'Churned').astype(int)

In [15]:
# Generate classification report using the test predictions
report_table = classification_report(test_data['CurrLifecycle_Churned'], test_data['PredLifecycle_Churned']) 
report_dict = classification_report(test_data['CurrLifecycle_Churned'], test_data['PredLifecycle_Churned'], output_dict=True) 

In [16]:
print(report_table)

              precision    recall  f1-score   support

           0       1.00      0.81      0.89     26023
           1       0.58      1.00      0.74      6984

    accuracy                           0.85     33007
   macro avg       0.79      0.90      0.81     33007
weighted avg       0.91      0.85      0.86     33007



In [17]:
# Check format in dictionary
print(report_dict)

{'0': {'precision': 1.0, 'recall': 0.8077854205894786, 'f1-score': 0.8936740073123034, 'support': 26023.0}, '1': {'precision': 0.5826797930919406, 'recall': 1.0, 'f1-score': 0.7363205060622035, 'support': 6984.0}, 'accuracy': 0.8484563880389009, 'macro avg': {'precision': 0.7913398965459704, 'recall': 0.9038927102947393, 'f1-score': 0.8149972566872534, 'support': 33007.0}, 'weighted avg': {'precision': 0.9116985995381015, 'recall': 0.8484563880389009, 'f1-score': 0.8603793470059836, 'support': 33007.0}}


In [31]:
# Save classification report as json
json_data = json.dumps(report_dict)

with open("Report_Dict.json", "w") as json_file:
    json_file.write(json_data)

In [18]:
# Filter out the existing active customers from the test data
#test_drop_index = test_data[(test_data['CurrLifecycle_Dormant'] == 1) | (test_data['CurrLifecycle_Churned'] == 1)].index
#test_active_prediction = test_data.drop(test_drop_index) 

In [19]:
# Concat the train_data and test_data data
predicted_data = pd.concat([train_data, test_data], ignore_index=True)   

In [20]:
predicted_data.head()

,CustomerId,Gender,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,ChurnDate,MonthsInactive,TransactionFreq,...,CurrLifecycle_Reactivated,average_Active_proba,average_Reactivated_proba,average_Dormant_proba,average_Churned_proba,PredictedLifecycle,PredLifecycle_Active,PredLifecycle_Reactivated,PredLifecycle_Dormant,PredLifecycle_Churned
0,89356,Female,26,10,106970.610000,1,93718.38,NaN,NaN,61,...,1,0.998492,0.0,0.000891,0.000617,Active,1,0,0,0
1,29738,Male,27,2,154250.048593,2,119327.75,NaN,NaN,4,...,0,0.497436,0.0,0.251286,0.251277,Active,1,0,0,0
2,128595,Female,46,15,104356.880000,1,54869.05,2024-01-25,2.0,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,55683,Female,36,14,95059.020000,1,44420.18,NaN,NaN,12,...,0,0.665495,0.0,0.000024,0.334481,Active,1,0,0,0
4,85378,Male,57,7,130385.820000,1,130661.95,2023-08-09,8.0,0,...,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [21]:
# Save it as csv for the API file to read
predicted_data.to_csv("Predicted_Data.csv")

In [22]:
# Check format of dictionary
predicted_data.head().to_dict(orient='records')

[{'CustomerId': 89356,
  'Gender': 'Female',
  'Age': 26,
  'Tenure': 10,
  'Balance': 106970.61,
  'NumOfProducts': 1,
  'EstimatedSalary': 93718.38,
  'ChurnDate': nan,
  'MonthsInactive': nan,
  'TransactionFreq': 61,
  'TransactionAmt': 9313.56,
  'ServiceSupportFrequency': 18,
  'NPS': 0.0,
  'Education': 'A',
  'EmploymentStatus': 'Fulltime',
  'MaritalStatus': 'Married',
  'HousingStatus': 'owned',
  'Dependents': 3,
  'MarketingOffersAcceptance': 0.2378135740421388,
  'PaymentMethod': 'Debit',
  'CustomerSatisfaction': 5,
  'FeatureSatisfaction': 2,
  'SupportSatisfaction': 1,
  'FeatureSupportFrequency': 12,
  'LoanAmt': 22733.74,
  'IncomeSource': 'Employment',
  'Retention': 0.4468054143563848,
  'ChangeInBehaviourMkt': 1.1439521391656498,
  'ChangeInBehaviourCust': 0.8170695685059123,
  'PrevLifecycle': 'Dormant',
  'CurrLifecycle': 'Reactivated',
  'Happiness': 1,
  'Savings Savant': 48.27245804540967,
  'Digital Dynamos': 30.207305034550835,
  'Trustee Tribe': 21.52023692